In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [8]:
df=pd.read_csv('train.csv')

In [9]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [13]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [14]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [28]:
# impution tranformer
trnf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [29]:
#ohe
trnf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [30]:
#scaling
trnf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [31]:
#fature selction
trnf4=SelectKBest(score_func=chi2,k=6)

In [32]:
#decison tree
trnf5=DecisionTreeClassifier()

In [33]:
pipe=Pipeline([
    ('trnf1',trnf1),
    ('trnf2',trnf2),
    ('trnf3',trnf3),
    ('trnf4',trnf4),
    ('trnf5',trnf5)
])

In [34]:
#pipe=make_pipeline(trnf1,trnf2,trnf3,trnf4,trnf5)

In [37]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trnf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trnf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x000002CF24CAAA60>)),
                ('trnf5', DecisionTreeClassifier())])

In [36]:
from sklearn import set_config
set_config(display='diagram')

In [42]:
pipe.named_steps['trnf1'].transformers_[0][1].statistics_

array([29.49884615])

In [43]:
pipe.named_steps['trnf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [44]:
y_pred=pipe.predict(x_test)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

In [47]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=10,scoring='accuracy').mean()

0.6391431924882629

In [50]:
params={
    'trnf5__max_depth':[1,2,3,4,5,None]
}

In [52]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trnf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trnf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trnf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trnf4',
                                        SelectKBest(k=6,
                                                    score_func=<function chi2 at 0x000002CF24CAAA60>)),
                                       ('trnf5', DecisionTreeClassifier())]),
             param_grid={'trnf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [53]:
grid.best_score_

0.6391214419383433

In [54]:
grid.best_params_

{'trnf5__max_depth': 2}

In [56]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))